# Preprocessing raw data

So you've recorded data in the Epilepsy monitoring unit - congratulations! What now? How do we load that data into a format that plays nice with Python, preprocess that data, and save it in a re-usable way? 

The LFPAnalysis codebase  contains a lot of functions to help with this. You can always dig into what those functions are doing (I encourage this) but in the notebooks I will simply show you how to **use** these functions. 

## Import packages

First, import Python packages and code you'll need:

In [1]:
import numpy as np
import mne
import pandas as pd
import warnings 

# I only want to see warnings once
warnings.filterwarnings('ignore')

**ACTION**: If you have installed the LFPAnalysis you must append the local path!

In [2]:
import sys
sys.path.append('/Users/salmanqasim/Documents/GitRepos/LFPAnalysis')

Import the custom functions from LFPAnalysis that we want

In [3]:
from LFPAnalysis import lfp_preprocess_utils, sync_utils, analysis_utils

## IF USING YOUR OWN DATA: Pre-process (run 1x): 

In the pre-processing functions below, we: 

1. load the raw data (either a .edf file or a folder of .nlx files) into mne objects for use with the mne toolbox: https://mne.tools/stable/index.html.

2. load the localized electrode names from the .csv or .xlsx file listing their MNI coordinates into the mne object

3. filter and resample as necessary

4. re-reference 

**NOTE**: this notebook is meant to use the sample data, stored locally in the repo. This local data was pre-processed using the follow cell, which you will need to uncomment and modify according to your own data folders.

In [4]:
# for ix, subj_id in enumerate(subj_ids): 
#     site = subj_sites[ix] # where the data was recorded, e.g. 'MSSM', 'UI'
#     format = subj_formats[ix] # the format the data, e.g. 'edf', 'nlx'
    
#     print(f'Working on subj {subj_id}')
    
#     # Set paths
#     load_path = f'{base_dir}/projects/guLab/Salman/EMU/{subj_id}/neural/Day1'
#     elec_path = f'{base_dir}/projects/guLab/Salman/EMU/{subj_id}/anat/'
#     save_path = f'{base_dir}/projects/guLab/Salman/EphysAnalyses/{subj_id}/neural/Day1'
    
#     # Check if path exists for saving, and if not, make it
#     if not os.path.exists(save_path):
#         os.makedirs(save_path)

#     # electrode files could either be csv or excel
#     elec_files = glob(f'{elec_path}/*.csv') + glob(f'{elec_path}/*.xlsx')
#     # There should really only be one, so grab it with the zero-index 
#     elec_file = elec_files[0]

#     # Make MNE file
#     mne_data = lfp_preprocess_utils.make_mne(load_path=load_path, 
#                                              elec_path=elec_file,
#                                              format=format,
#                                              return_data=True,
#                                              site=site,
#                                              check_bad=False) # changed this to not annotate anything as bad 

#     # Save this data so that you don't need this step again:
#     mne_data.save(f'{save_path}/raw_ieeg.fif', overwrite=True)


If you are not using your own data (e.g. trying out the sample data) then you can just proceed from this point onwards. 

## Re-reference the data:

We re-reference the data to get rid of shared noise, cleaning the data to leave what we assume is local biological activity. 

Read more in [this nice chapter from Parish et al.](https://link.springer.com/chapter/10.1007/978-3-031-20910-9_28)

Here, note that we default to a bipolar configuration. This is my personal preference for my datasets but you have the freedom to use a white matter reference as well. I have not implemented a Laplacian or common-average referencing scheme yet. 

Note also that re-referencing requires modifying the electrode dataframes that contain the names and regions of each electrode. This is pretty easy in the bipolar montage but keep in mind if you implement a different method for this.

In [5]:
site = 'MSSM' # where the data was recorded, e.g. 'MSSM', 'UI'
format = 'edf' # the format the data, e.g. 'edf', 'nlx'
    
# Set load path
load_path = '/Users/salmanqasim/Documents/GitRepos/LFPAnalysis/data'

# Load electrode file 
elec_file = f'{load_path}/sample_labels.xlsx'

# Make MNE file
mne_data = mne.io.read_raw_fif(f'{load_path}/sample_ieeg.fif', preload=True)


# Re-reference neural data
mne_data_reref = lfp_preprocess_utils.ref_mne(mne_data=mne_data, 
                                              elec_path=elec_file, 
                                              method='bipolar', 
                                              site=site)

# Save this data so that you don't need this step again:
mne_data_reref.save(f'{load_path}/sample_ieeg_bp.fif', overwrite=True)

# Should also save out re-referenced elec_file: 

elec_data = lfp_preprocess_utils.load_elec(elec_file)
anode_list = [x.split('-')[0] for x in mne_data_reref.ch_names]
elec_df = elec_data[elec_data.label.str.lower().isin(anode_list)]
elec_df['label'] =  elec_df.label.apply(lambda x: [a for a in mne_data_reref.ch_names if str(x).lower() in a.split('-')[0]][0])

# Add region to the data frame 

manual_col = [col for col in elec_df.columns if 'manual' in col.lower()][0]
all_regions = [] 
for chan_name in elec_df.label.unique():
    elec_region = analysis_utils.select_rois_picks(elec_df, chan_name, manual_col=manual_col)
    all_regions.append(elec_region) 

elec_df['salman_region'] = all_regions
elec_df['hemisphere'] = elec_df.label.apply(lambda x: x[0])

elec_df.to_csv(f'{load_path}/sample_labels_bp', index=False)
            


Opening raw data file /Users/salmanqasim/Documents/GitRepos/LFPAnalysis/data/sample_ieeg.fif...


Isotrak not found


    Range : 0 ... 394061 =      0.000 ...   788.122 secs


Ready.


Reading 0 ... 394061  =      0.000 ...   788.122 secs...


...

/Users/salmanqasim/Documents/GitRepos/LFPAnalysis/LFPAnalysis/../data/YBA_ROI_labelled.xlsx
/Users/salmanqasim/Documents/GitRepos/LFPAnalysis/LFPAnalysis/../data/YBA_ROI_labelled.xlsx


/Users/salmanqasim/Documents/GitRepos/LFPAnalysis/LFPAnalysis/../data/YBA_ROI_labelled.xlsx


/Users/salmanqasim/Documents/GitRepos/LFPAnalysis/LFPAnalysis/../data/YBA_ROI_labelled.xlsx


/Users/salmanqasim/Documents/GitRepos/LFPAnalysis/LFPAnalysis/../data/YBA_ROI_labelled.xlsx
/Users/salmanqasim/Documents/GitRepos/LFPAnalysis/LFPAnalysis/../data/YBA_ROI_labelled.xlsx


/Users/salmanqasim/Documents/GitRepos/LFPAnalysis/LFPAnalysis/../data/YBA_ROI_labelled.xlsx
/Users/salmanqasim/Documents/GitRepos/LFPAnalysis/LFPAnalysis/../data/YBA_ROI_labelled.xlsx


 - mne_data: a Raw mne object, where the data has been loaded, filtered for line noise, parsed for different data types, and resampled if necessary. 
 
 - mne_data_reref: an mne object containing re-referenced data (either white matter or bipolar)

## NOW look at the data to manually remove channels: 

After bipolar referencing: 

In [6]:
# %matplotlib notebook 

In [7]:
# # Scroll up/down and left/right using your keyboard. CLICK on a channel to turn it 'grey' and mark as a 'bad' channel. 
# # If you click a grey channel again it will unmark it. 

# mne_data_reref = mne.io.read_raw_fif(f'{load_path}/sample_ieeg_bp.fif', preload=True)
# fig = mne_data_reref.plot(start=0, duration=120, n_channels=30, 
#                       scalings=mne_data_reref._data.max()/30
#                      )

In [8]:
# # ALSO look at the power spectra! 
# # You can click on channels here to identify them, and go back to the viz above to mark them as noise if need be

# mne_data_reref.compute_psd().plot()

If you have ran the preprocessing above, load the data instead: 


In [9]:
load_path = '/Users/salmanqasim/Documents/GitRepos/LFPAnalysis/data'

elec_file = f'{load_path}/sample_labels.xlsx'

elec_data = lfp_preprocess_utils.load_elec(elec_file)

mne_data_reref = mne.io.read_raw_fif(f'{load_path}/sample_ieeg_bp.fif', preload=True)

photodiode_data = mne.io.read_raw_fif(f'{load_path}/sample_photodiode.fif', preload=True)

Opening raw data file /Users/salmanqasim/Documents/GitRepos/LFPAnalysis/data/sample_ieeg_bp.fif...


    Range : 0 ... 394061 =      0.000 ...   788.122 secs


Ready.


Reading 0 ... 394061  =      0.000 ...   788.122 secs...


Opening raw data file /Users/salmanqasim/Documents/GitRepos/LFPAnalysis/data/sample_photodiode.fif...


Isotrak not found


    Range : 0 ... 807039 =      0.000 ...   788.124 secs


Ready.


Reading 0 ... 807039  =      0.000 ...   788.124 secs...


 - mne_dict: a dictionary containing all of your subjects' re-referenced mne data 
 
 - photodiode_dict: a dictionary containing all of your subjects' photodiode data 
 
 - elec_dict: a dictionary containing the paths to your subjects' electrode data 

## Sync behavioral and neural data

Here, we perform a critical step: computing the time offset between the computer that recorded the neural data and the laptop that featured the experiment. 

The function here only requires a **subset** of detected sync signals (i.e. photodiode deflections) to be detected to successfully compute this offset. 

First, you may need to MANUALLY clean the photodiode signal if the recording quality is poor. Load it, plot it, and try to isolate/amplify the pulses. 

In [10]:
time_df = pd.read_csv(f'{load_path}/sample_ts.csv')
# Load in the timestamps pertaining to your sync. If your task had a square pop up, for example, grab the times for that square's appearance from the behavioral logs.
# Below, I do this for my own task's Psychopy output, but yours is probably coded differently. 

# Synchronize to the photodiode or whatever your neural sync signal is
height = 1
windSize = 15
smoothSize = 11

slope, offset = sync_utils.synchronize_data(time_df['beh_ts'].values, 
                                            photodiode_data, 
                                            smoothSize=smoothSize, windSize=windSize, height=height)

print(slope)
print(offset)

28 blocks
.

................

.

..

..

..

...

.

found matches for 150 of 428 pulses
0.9999892560991055
-228.2080128605071


 - slopes: a dictionary containing the slopes (should be ~ 1) for each subject
 - offsets: a dictionary containing the offsets for each subject

## Load your behavioral data

You probably have a separate notebook for processing the behavioral data for your task. Load the processed dataframe here:

In [11]:
behav_data = pd.read_csv(f'{load_path}/sample_beh.csv')

In [12]:
behav_data.head(5)[['trials', 'feedback_start', 'baseline_start']]

,trials,feedback_start,baseline_start
0,1,243.239158,244.929025
1,2,248.344187,250.043187
2,3,254.083059,255.790670
3,4,258.148220,259.838892
4,5,261.943712,263.620631


## Make epochs

Make epochs and remove IEDs. Currently just doing this for one example period - when subjects receive feedback. 

Notes: 

- I also segment a baseline period for every event of interest. 

- I apply a buffer period of 1.0 seconds - this will be helpful when we compute spectrograms later. 

- The IED count for every channel is added to the epoch metadata

(I'm a little dumb, so my baseline is a fixation cross AFTER the trial, rather than before. A bit silly if you ask me.) 

In [13]:
# set some windows of interest 

buf = 1.0 # this is the buffer before and after that we use to limit edge effects for TFRs

IED_args = {'peak_thresh':5,
           'closeness_thresh':0.25, 
           'width_thresh':0.2}

evs = {'feedback_start': [-0.5, 1.5],
       'baseline_start': [0, 0.75]}

load_path = '/Users/salmanqasim/Documents/GitRepos/LFPAnalysis/data'

epochs_all_evs = {f'{x}': np.nan for x in evs}
for event in evs.keys():
    pre = evs[event][0]
    post = evs[event][1]
    fixed_baseline = None
    behav_times = behav_data[event]

    # THE following function will now SAVE out dataframes that indicate IED and artifact time points in your data
    epochs = lfp_preprocess_utils.make_epochs(load_path=f'{load_path}/sample_ieeg_bp.fif', 
                                              slope=slope, offset=offset, 
                                              behav_name=event, behav_times=behav_times,
                                              ev_start_s=pre, ev_end_s=post, buf_s=1, downsamp_factor=None, IED_args=IED_args, detrend=0)


    epochs_all_evs[event] = epochs
    epochs_all_evs[event].save(f'{load_path}/sample_{event}-epo.fif', overwrite=True)


Opening raw data file /Users/salmanqasim/Documents/GitRepos/LFPAnalysis/data/sample_ieeg_bp.fif...


    Range : 0 ... 394061 =      0.000 ...   788.122 secs


Ready.


Reading 0 ... 394061  =      0.000 ...   788.122 secs...


Filtering raw data in 1 contiguous segment


...

0 projection items activated


Using data from preloaded Raw for 80 events and 1376 original time points ...


0 bad epochs dropped


Overwriting existing file.


Overwriting existing file.


 - epochs_all_evs: dictionary containing the epochs for all of your subjects' re-referenced data

Plot and examine the epochs if you'd like:

In [14]:
# %matplotlib notebook
# fig = epochs_all_subjs_all_evs['MS007']['feedback_start'].plot(n_epochs=10, n_channels=10)

In [15]:
# # Need this following line to save the annotations to the epochs object 
# fig.fake_keypress('a')

## Where do I go from here? 

At this point, you've successfuly pre-processed your iEEG data and sliced it around your timepoints of interest. These epochs are going to be the currency for many of your subsequent analyses, so make sure you TRUST THEM before proceeding to the other notebooks for analyses. 

From here, you can move on to the:

1. FOOOF: a notebook for computing power-spectra across trials and fitting their peaks 

2. TFRPlotsAndStatistics: a notebook for computing time-frequency spectra (trial-level), and computing several different statistics or simply saving the data out in dataframes. 

3. OscillationDetection(BOSC): a notebook for computing sliding burst detection and saving the data out in dataframes 

4. TimeResolvedRegression: a notebook for computing regression analysis at each timepoint of a timeseries. TFR-extracted band power is used as example. 

5. ConnectivityAnalysis: a notebook for computing different synchrony measures between electrodes. 